In [1]:
import pickle,os,sys,gc
import pandas as pd
import numpy as np
import ssdeep
import glob
from tqdm import tqdm
import functools,operator
from datetime import datetime
from sklearn.metrics import *
from sklearn.metrics.pairwise import *
from collections import Counter
# import matplotlib相關套件
import matplotlib.pyplot as plt
# import字型管理套件
from matplotlib.font_manager import FontProperties
from matplotlib import pyplot
from operator import itemgetter
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [2]:
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
date_li.reverse()
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
# protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']

isp_li.reverse()
file_name = "_".join(sorted(date_li)) #date_li 改!!
min_date = str(min(date_li))
max_date = str(max(date_li))


In [3]:
# pkt_sess_df = pd.DataFrame(columns=isp_li,index=date_li)
# country_count_dict = {}
# protocol_count_dict = {}
# tri_country_df = pd.DataFrame()
# port_dist_df = pd.DataFrame()
tri_country_df = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/IrelandCnTw_beforepreprocess_dist_reverse.pkl','rb'))
tri_country_df

,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
20200112_台灣碩網_China,"[554, 554, 80, 22, 22, 80, 80, 80, 80, 80, 80,...","[11, 6, 36, 16, 13, 6, 7, 6, 6, 8, 12, 13, 12,...","[1015, 295, 14219, 252, 913, 5, 268, 273, 273,..."
20200112_台灣碩網_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[19, 22, 17, 7, 11, 22, 16, 13, 13, 16, 13, 10...","[1448, 1152, 936, 12, 1092, 1224, 1116, 1176, ..."
20200112_台灣碩網_Taiwan,"[445, 445, 445, 445, 445, 445, 8080, 3389, 445...","[9, 8, 8, 8, 7, 6, 9, 6, 8, 5, 5, 5, 5, 5, 5, ...","[455, 455, 364, 455, 222, 364, 412, 19, 364, 1..."
20200112_台灣基礎開發_China,"[445, 445, 22, 22, 22, 445, 445, 445, 445, 445...","[6, 7, 8, 5, 12, 52, 6, 14, 13, 8, 12, 6, 7, 7...","[191, 191, 541, 23, 1151, 64075, 364, 729, 866..."
20200112_台灣基礎開發_Ireland,"[80, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[23, 24, 16, 15, 12, 12, 11, 19, 12, 15, 12, 1...","[15296, 1804, 1448, 1412, 1176, 1176, 1092, 14..."
20200112_台灣基礎開發_Taiwan,"[445, 445, 445, 445, 445, 445, 80, 110, 445, 1...","[16, 12, 55, 8, 8, 8, 7, 11, 52, 6, 6, 5, 6, 6...","[907, 455, 64075, 455, 222, 364, 53, 71, 63801..."
20200112_台灣之星_China,"[22, 80, 22, 22, 22, 22, 22, 22, 22, 80, 80, 8...","[12, 6, 14, 12, 12, 13, 10, 12, 12, 6, 5, 4, 1...","[745, 89, 909, 745, 745, 782, 105, 745, 745, 2..."
20200112_台灣之星_Ireland,"[445, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, ...","[16, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1...","[408, 1312, 1312, 1312, 1312, 1312, 1312, 1312..."
20200112_台灣之星_Taiwan,"[445, 445, 1433, 1433, 1433, 1433, 1433, 1433,...","[10, 8, 6, 5, 7, 5, 8, 6, 6, 5, 6, 4, 6, 6, 9,...","[537, 191, 192, 192, 224, 194, 224, 230, 228, ..."
20200112_凱擘_China,"[445, 445, 8080, 554, 554, 21, 21, 21, 21, 21,...","[8, 8, 5, 6, 7, 25, 24, 23, 23, 25, 24, 5, 4, ...","[364, 364, 304, 295, 295, 171, 168, 134, 132, ..."


In [5]:

# take_columns = ['session_i_tt_packet','session_o_tt_packet']

# counter = 0
for da in tqdm(date_li):
    for isp in isp_li:
        if da+'_'+isp in processed_li_rev:
            continue #之前有一部分已經跑過了        
        print("=== Now is about to run:",da,isp,"===")
        gc.collect()
        dir_path = "/mnt/Raid160TB/pcap_inter/"+str(da[:4])+'_'+str(da[4:6])+'_'+str(da[6:])+"/"+isp+"/intermeidate_data/"
        saved_path = dir_path+"df2_tuples_"+isp+".pkl"
        df2,pickle_dir,picture_dir = pickle.load(open(saved_path,'rb')) 
        
        temp1_master = df2[['country','tcp_dstport','session_i_tt_packet','tcp_i_tt_payload_length']]
        temp1 = temp1_master.groupby(['country'])
        temp1 = temp1.aggregate(list)
        temp11 = temp1[temp1.index.isin(['China','Taiwan','Ireland'])] #指定需要的國家
        t_index = temp11.index.tolist()
        t_index = [da+'_'+isp+'_'+ind for ind in t_index]
        temp11.index = t_index
        tri_country_df = tri_country_df.append(temp11)
        
#         temp1_master = df2[['country','tcp_dstport','session_i_tt_packet','session_o_tt_packet','tcp_i_tt_payload_length','tcp_o_tt_payload_length','frame_i_max_protocols','frame_o_max_protocols','tcp_i_payload_list','tcp_o_payload_list']]
#         temp2 = temp1_master.groupby(['tcp_dstport'])
#         temp2 = temp2.aggregate(list)
#         t_index = temp2.index.tolist()
#         t_index = [da+'_'+isp+'_'+str(ind) for ind in t_index]
#         temp2.index = t_index
#         port_dist_df = port_dist_df.append(temp2)
        gc.collect()
        #checkpoint
        pickle.dump(obj=tri_country_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/IrelandCnTw_beforepreprocess_dist_reverse.pkl','wb'))
#         pickle.dump(obj=port_dist_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/all_port_beforepreprocess_dist_reverse.pkl','wb'))
#         break
#     break
# print(df2.columns) #tcp_dstport、session_duration、session_i_tt_packet、tcp_i_tt_payload_length、
# df2
tri_country_df

  0%|          | 0/7 [00:00<?, ?it/s]

=== Now is about to run: 20200112 亞太電信 ===
=== Now is about to run: 20200112 中嘉寬頻 ===


 14%|█▍        | 1/7 [01:18<07:49, 78.23s/it]

=== Now is about to run: 20200111 台灣碩網 ===
=== Now is about to run: 20200111 台灣基礎開發 ===
=== Now is about to run: 20200111 台灣之星 ===
=== Now is about to run: 20200111 凱擘 ===
=== Now is about to run: 20200111 中華電信 ===
=== Now is about to run: 20200111 台灣大哥大 ===
=== Now is about to run: 20200111 台灣固網 ===
=== Now is about to run: 20200111 台固媒體 ===
=== Now is about to run: 20200111 亞太電信 ===
=== Now is about to run: 20200111 中嘉寬頻 ===


 29%|██▊       | 2/7 [5:33:57<16:21:04, 11772.87s/it]

...

=== Now is about to run: 20200107 台灣碩網 ===
=== Now is about to run: 20200107 台灣基礎開發 ===
=== Now is about to run: 20200107 台灣之星 ===
=== Now is about to run: 20200107 凱擘 ===
=== Now is about to run: 20200107 中華電信 ===
=== Now is about to run: 20200107 台灣大哥大 ===
=== Now is about to run: 20200107 台灣固網 ===
=== Now is about to run: 20200107 台固媒體 ===
=== Now is about to run: 20200107 亞太電信 ===
=== Now is about to run: 20200107 中嘉寬頻 ===


 86%|████████▌ | 6/7 [15:06:58<2:40:23, 9623.27s/it]

=== Now is about to run: 20200106 台灣碩網 ===
=== Now is about to run: 20200106 台灣基礎開發 ===
=== Now is about to run: 20200106 台灣之星 ===
=== Now is about to run: 20200106 凱擘 ===
=== Now is about to run: 20200106 中華電信 ===
=== Now is about to run: 20200106 台灣大哥大 ===
=== Now is about to run: 20200106 台灣固網 ===
=== Now is about to run: 20200106 台固媒體 ===
=== Now is about to run: 20200106 亞太電信 ===
=== Now is about to run: 20200106 中嘉寬頻 ===


100%|██████████| 7/7 [17:00:34<00:00, 8747.73s/it]  


,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
20200112_台灣碩網_China,"[554, 554, 80, 22, 22, 80, 80, 80, 80, 80, 80,...","[11, 6, 36, 16, 13, 6, 7, 6, 6, 8, 12, 13, 12,...","[1015, 295, 14219, 252, 913, 5, 268, 273, 273,..."
20200112_台灣碩網_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[19, 22, 17, 7, 11, 22, 16, 13, 13, 16, 13, 10...","[1448, 1152, 936, 12, 1092, 1224, 1116, 1176, ..."
20200112_台灣碩網_Taiwan,"[445, 445, 445, 445, 445, 445, 8080, 3389, 445...","[9, 8, 8, 8, 7, 6, 9, 6, 8, 5, 5, 5, 5, 5, 5, ...","[455, 455, 364, 455, 222, 364, 412, 19, 364, 1..."
20200112_台灣基礎開發_China,"[445, 445, 22, 22, 22, 445, 445, 445, 445, 445...","[6, 7, 8, 5, 12, 52, 6, 14, 13, 8, 12, 6, 7, 7...","[191, 191, 541, 23, 1151, 64075, 364, 729, 866..."
20200112_台灣基礎開發_Ireland,"[80, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[23, 24, 16, 15, 12, 12, 11, 19, 12, 15, 12, 1...","[15296, 1804, 1448, 1412, 1176, 1176, 1092, 14..."
...,...,...,...
20200106_亞太電信_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[6, 6, 6, 5, 11, 11, 13, 23, 15, 20, 54, 16, 1...","[12, 12, 12, 12, 1092, 956, 956, 960, 1412, 13..."
20200106_亞太電信_Taiwan,"[1433, 445, 445, 1433, 1433, 1433, 1433, 1433,...","[10, 8, 6, 8, 8, 13, 6, 10, 11, 10, 6, 6, 6, 9...","[575, 455, 267, 388, 378, 570, 184, 426, 582, ..."
20200106_中嘉寬頻_China,"[1433, 1433, 1433, 1433, 1433, 1433, 1433, 143...","[6, 8, 6, 7, 6, 7, 8, 8, 7, 8, 8, 7, 6, 6, 6, ...","[300, 398, 347, 400, 351, 349, 351, 398, 408, ..."
20200106_中嘉寬頻_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[16, 15, 14, 14, 14, 14, 14, 31, 14, 14, 14, 1...","[713, 713, 1312, 1312, 1312, 1312, 1312, 3264,..."


In [6]:
processed_li = tri_country_df.index.tolist()
processed_li_rev = ["_".join(x.split('_')[:2]) for x in processed_li]
processed_li_rev = list(set(processed_li_rev))
processed_li_rev

['20200109_凱擘',
 '20200109_中華電信',
 '20200109_台灣固網',
 '20200108_台灣碩網',
 '20200108_亞太電信',
 '20200112_台灣基礎開發',
 '20200111_台灣大哥大',
 '20200107_台灣之星',
 '20200110_台固媒體',
 '20200112_台固媒體',
 '20200106_凱擘',
 '20200111_台灣之星',
 '20200112_亞太電信',
 '20200107_台灣固網',
 '20200106_台灣固網',
 '20200110_台灣之星',
 '20200111_中華電信',
 '20200110_凱擘',
 '20200112_中華電信',
 '20200108_中嘉寬頻',
 '20200109_台灣基礎開發',
 '20200110_亞太電信',
 '20200111_亞太電信',
 '20200112_台灣大哥大',
 '20200111_台灣基礎開發',
 '20200111_中嘉寬頻',
 '20200110_中嘉寬頻',
 '20200109_台灣碩網',
 '20200109_台灣大哥大',
 '20200107_凱擘',
 '20200108_台灣固網',
 '20200106_台灣之星',
 '20200108_台灣大哥大',
 '20200112_凱擘',
 '20200112_中嘉寬頻',
 '20200110_台灣基礎開發',
 '20200110_台灣大哥大',
 '20200111_凱擘',
 '20200108_台固媒體',
 '20200109_台固媒體',
 '20200108_台灣基礎開發',
 '20200107_台灣大哥大',
 '20200108_中華電信',
 '20200107_台灣碩網',
 '20200108_台灣之星',
 '20200106_中華電信',
 '20200109_台灣之星',
 '20200109_亞太電信',
 '20200110_台灣固網',
 '20200111_台灣固網',
 '20200107_中華電信',
 '20200112_台灣固網',
 '20200107_台固媒體',
 '20200107_台灣基礎開發',
 '20200112_台灣之星',
 '20

In [19]:
temp1 = df2[['country','tcp_dstport','session_i_tt_packet','tcp_i_tt_payload_length']]
temp1 = temp1.groupby(['country'])
temp1 = temp1.aggregate(list)
temp1
# df2[df2.country == 'China'] #Ireland、China、Taiwan

,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
country,,,
Afghanistan,[80],[177],[185]
Albania,"[8080, 80, 80, 80, 8080, 80, 8080, 80, 80, 80,...","[6, 7, 21, 11, 6, 69, 6, 7, 6, 22, 24, 242, 36...","[185, 186, 13909, 192, 192, 14533, 187, 185, 1..."
Algeria,"[445, 445, 445, 445, 445, 445, 445, 445, 445, ...","[6, 6, 6, 6, 8, 8, 13, 54, 8, 6, 6, 8, 6, 8, 8...","[364, 364, 364, 364, 455, 455, 866, 63938, 455..."
Angola,[22],[18],[1175]
Antigua and Barbuda,"[22, 22, 22]","[17, 17, 18]","[1175, 1175, 1079]"
...,...,...,...
Vietnam,"[445, 2222, 2222, 445, 2222, 2222, 22, 22, 22,...","[12, 13, 13, 6, 13, 13, 13, 14, 8, 14, 40, 10,...","[592, 713, 713, 364, 713, 713, 729, 932, 651, ..."
Yemen,"[445, 445, 445, 445, 445, 445, 445, 445, 445, ...","[4, 8, 8, 6, 8, 8, 8, 8, 8, 8, 13, 8, 18, 8, 6...","[191, 364, 455, 364, 455, 455, 442, 364, 455, ..."
Zambia,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[13, 13, 13, 16, 12, 13, 12, 13, 12, 15, 13, 1...","[713, 713, 713, 713, 745, 713, 745, 713, 745, ..."


In [28]:
temp11 = temp1[temp1.index.isin(['China','Taiwan','Ireland'])]
t_index = temp11.index.tolist()
t_index = [da+'_'+isp+'_'+ind for ind in t_index]
temp11.index = t_index
temp11

,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
20200106_中嘉寬頻China,"[1433, 1433, 1433, 1433, 1433, 1433, 1433, 143...","[6, 8, 6, 7, 6, 7, 8, 8, 7, 8, 8, 7, 6, 6, 6, ...","[300, 398, 347, 400, 351, 349, 351, 398, 408, ..."
20200106_中嘉寬頻Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[16, 15, 14, 14, 14, 14, 14, 31, 14, 14, 14, 1...","[713, 713, 1312, 1312, 1312, 1312, 1312, 3264,..."
20200106_中嘉寬頻Taiwan,"[445, 21, 21, 21, 21, 22, 22, 2222, 2222, 2222...","[56, 7, 7, 7, 7, 14, 14, 14, 16, 17, 16, 15, 1...","[63938, 33, 25, 31, 29, 886, 886, 886, 886, 88..."


In [46]:
temp1_master = df2[['country','tcp_dstport','session_i_tt_packet','session_o_tt_packet','tcp_i_tt_payload_length','tcp_o_tt_payload_length','frame_i_max_protocols','frame_o_max_protocols','tcp_i_payload_list','tcp_o_payload_list']]
temp2 = temp1_master.groupby(['tcp_dstport'])
temp2 = temp2.aggregate(list)
t_index = temp2.index.tolist()
t_index = [da+'_'+isp+'_'+str(ind) for ind in t_index]
temp2.index = t_index
temp2

,country,session_i_tt_packet,session_o_tt_packet,tcp_i_tt_payload_length,tcp_o_tt_payload_length,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list
20200106_中嘉寬頻_21,"[United States, Taiwan, Taiwan, Taiwan, Taiwan...","[6, 7, 7, 7, 7, 7, 7, 7, 8, 11, 11, 11, 5, 8, ...","[6, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10...","[10, 33, 25, 31, 29, 24, 33, 28, 41, 34, 37, 3...","[51, 113, 113, 113, 113, 113, 113, 113, 182, 1...","[eth:ethertype:ip:tcp:ftp, eth:ethertype:ip:tc...","[eth:ethertype:ip:tcp:ftp, eth:ethertype:ip:tc...","[[[3:2f0/9B9En:2f0/j9En, 1578249914.441039000,...",[[[3:QCWVSsTQCfyNTQ/OP/1Tpv9duESXjm:QCSSsTbJmd...
20200106_中嘉寬頻_22,"[Thailand, South Korea, United States, China, ...","[13, 13, 13, 13, 13, 14, 14, 10, 13, 14, 14, 1...","[15, 15, 15, 15, 15, 15, 15, 16, 15, 15, 19, 1...","[729, 713, 713, 713, 729, 886, 729, 509, 713, ...","[2092, 2092, 2092, 2092, 2092, 2044, 2092, 486...","[eth:ethertype:ip:tcp:ssh, eth:ethertype:ip:tc...","[eth:ethertype:ip:tcp:ssh, eth:ethertype:ip:tc...","[[[3:DfQMPmfK7PATaupm:DPQKtupm, 1578254105.213...",[[[3:DfQMPmflfTAfQW4Y7wUZIr6drA60vA6Eia1rm:DPQ...
20200106_中嘉寬頻_23,"[Romania, Romania, Romania, Romania, Romania, ...","[6, 6, 6, 6, 13, 6, 6, 6, 13, 6, 10, 6, 6, 5, ...","[5, 5, 5, 5, 7, 5, 5, 5, 6, 4, 6, 5, 4, 4, 5, ...","[33, 33, 4, 4, 24, 4, 4, 4, 954, 8, 16, 33, 48...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[eth:ethertype:ip:tcp:telnet, eth:ethertype:ip...","[eth:ethertype:ip:tcp:telnet, eth:ethertype:ip...",[[[3:3D/UdfrzC5SW/J2DrV7/UCVfFfrfVJDd/:gdCf+ra...,"[[[3:UNT2CIQ7A4fTxx18uZhEn:UgNIrfdxSiEn, 15782..."
20200106_中嘉寬頻_25,"[Vietnam, China, Sweden, Poland, Poland, Polan...","[40, 7, 6, 6, 10, 12, 8, 5, 6, 6, 7, 8, 7, 7, ...","[22, 8, 5, 8, 14, 16, 12, 6, 8, 8, 10, 11, 10,...","[32178, 15, 27, 31, 35, 35, 35, 11, 32, 32, 29...","[135, 54, 40, 54, 64, 64, 64, 38, 54, 54, 56, ...",[eth:ethertype:ip:tcp:smtp:imf:data-text-lines...,"[eth:ethertype:ip:tcp:smtp, eth:ethertype:ip:t...",[[[3:RlhfRcfR6P+5EfRGYdfqIW+aBfu:RlJRe6Pd7qFpB...,"[[[3:QCWVSsTQCfyNTQ97E:QCSSsTbJpE, 1578325360...."
20200106_中嘉寬頻_42,"[Russia, Russia, Russia, Russia, Russia, Germa...","[8, 8, 8, 8, 8, 8, 5, 8, 8, 8, 5, 5, 8, 8, 8, ...","[8, 7, 8, 8, 8, 7, 4, 7, 7, 7, 4, 5, 8, 8, 8, ...","[465, 465, 465, 465, 465, 465, 47, 465, 465, 4...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",[[[3:OXXXL1Vf1V1V1V1RYP/LfIQfdplr+M5DcEYVV91v/...,"[[[3:Sn:Sn, 1578284678.743698000, 2]], [[3:Sn:..."
20200106_中嘉寬頻_80,"[Thailand, China, China, India, Hong Kong, Ind...","[27, 38, 3473, 7, 6, 35, 5, 36, 122, 7, 6, 10,...","[16, 42, 2521, 6, 5, 29, 5, 22, 80, 6, 5, 5, 4...","[14986, 18596, 15544, 185, 256, 14419, 232, 14...","[14280, 48580, 17593647, 96, 446, 85432, 446, ...","[eth:ethertype:ip:tcp:http:urlencoded-form, et...","[eth:ethertype:ip:tcp:http:data-text-lines, et...",[[[24:uAawYD5K/S18YH+w18YHxwdN04adN/QDdlUX18YH...,[[[96:DVb4NF1Cp4QuQxr9sHjqozJnsnSnrnb01gH1FUyY...
20200106_中嘉寬頻_110,"[United States, United States, Romania, Nether...","[7, 7, 7, 7, 8, 7, 9, 7, 7, 7, 7, 5, 8, 12, 7,...","[6, 6, 6, 6, 6, 6, 10, 6, 6, 6, 6, 5, 7, 11, 6...","[10, 6, 18, 204, 18, 18, 71, 18, 18, 18, 18, 6...","[18, 18, 18, 18, 18, 18, 164, 18, 18, 18, 18, ...","[eth:ethertype:ip:tcp:pop, eth:ethertype:ip:tc...","[eth:ethertype:ip:tcp:pop:imf, eth:ethertype:i...","[[[3:2f0/9B9En:2f0/j9En, 1578240246.977626000,...","[[[3:QCWVRBf5jWfPSXjm:QCScfqa, 1578240246.8144..."
20200106_中嘉寬頻_111,"[Vietnam, Vietnam, Vietnam, Vietnam, Vietnam, ...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[120, 120, 120, 120, 120, 120, 120, 120, 120, ...","[64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 6...","[eth:ethertype:ip:tcp:rpc:portmap, eth:etherty...","[eth:ethertype:ip:tcp:rpc:portmap, eth:etherty...",[[[3:PVVfXWdu469V1V1V1V1V1VP/B1XV1VP/vV1VYvV1V...,[[[3:PVVf

In [26]:
Counter(temp1.loc['China']['tcp_dstport'])

Counter({1433: 69164,
         2222: 39694,
         80: 19158,
         22: 158906,
         445: 4017,
         3389: 1733,
         8080: 297,
         25: 131,
         21: 480,
         139: 2388,
         3306: 899,
         1023: 113,
         9999: 18,
         135: 70,
         443: 60,
         1025: 112,
         2954: 61,
         587: 14,
         3128: 16,
         1080: 5,
         143: 3,
         23: 2})

In [ ]:
dir_path = "/mnt/Raid160TB/pcap_inter/"+str(date[:4])+'_'+str(date[4:6])+'_'+str(date[6:])+"/"+isp+"/intermeidate_data/"
df2,pickle_dir,picture_dir = pickle.load(open(dir_path+"df2_tuples_"+isp+".pkl",'rb'))
df2

tri_country_df

計算這三個國家的port每天是再做甚麼

In [8]:
tri_country_df

,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
20200112_台灣碩網_China,"[554, 554, 80, 22, 22, 80, 80, 80, 80, 80, 80,...","[11, 6, 36, 16, 13, 6, 7, 6, 6, 8, 12, 13, 12,...","[1015, 295, 14219, 252, 913, 5, 268, 273, 273,..."
20200112_台灣碩網_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[19, 22, 17, 7, 11, 22, 16, 13, 13, 16, 13, 10...","[1448, 1152, 936, 12, 1092, 1224, 1116, 1176, ..."
20200112_台灣碩網_Taiwan,"[445, 445, 445, 445, 445, 445, 8080, 3389, 445...","[9, 8, 8, 8, 7, 6, 9, 6, 8, 5, 5, 5, 5, 5, 5, ...","[455, 455, 364, 455, 222, 364, 412, 19, 364, 1..."
20200112_台灣基礎開發_China,"[445, 445, 22, 22, 22, 445, 445, 445, 445, 445...","[6, 7, 8, 5, 12, 52, 6, 14, 13, 8, 12, 6, 7, 7...","[191, 191, 541, 23, 1151, 64075, 364, 729, 866..."
20200112_台灣基礎開發_Ireland,"[80, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[23, 24, 16, 15, 12, 12, 11, 19, 12, 15, 12, 1...","[15296, 1804, 1448, 1412, 1176, 1176, 1092, 14..."
...,...,...,...
20200106_亞太電信_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[6, 6, 6, 5, 11, 11, 13, 23, 15, 20, 54, 16, 1...","[12, 12, 12, 12, 1092, 956, 956, 960, 1412, 13..."
20200106_亞太電信_Taiwan,"[1433, 445, 445, 1433, 1433, 1433, 1433, 1433,...","[10, 8, 6, 8, 8, 13, 6, 10, 11, 10, 6, 6, 6, 9...","[575, 455, 267, 388, 378, 570, 184, 426, 582, ..."
20200106_中嘉寬頻_China,"[1433, 1433, 1433, 1433, 1433, 1433, 1433, 143...","[6, 8, 6, 7, 6, 7, 8, 8, 7, 8, 8, 7, 6, 6, 6, ...","[300, 398, 347, 400, 351, 349, 351, 398, 408, ..."
20200106_中嘉寬頻_Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[16, 15, 14, 14, 14, 14, 14, 31, 14, 14, 14, 1...","[713, 713, 1312, 1312, 1312, 1312, 1312, 3264,..."


# 計算沒payload的port在做甚麼

In [9]:
port_df = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/all_port_nopayload_dist.pkl','rb'))
port_df

,country,session_i_tt_packet,session_o_tt_packet,frame_i_max_protocols,frame_o_max_protocols
20200106_中嘉寬頻_0,"[Denmark, Netherlands]","[1, 1]","[1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp]"
20200106_中嘉寬頻_1,"[France, France, Russia, Russia, Russia, Franc...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_中嘉寬頻_2,"[Russia, Russia, Russia, Denmark]","[2, 2, 2, 1]","[1, 1, 1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_中嘉寬頻_3,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_中嘉寬頻_4,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
...,...,...,...,...,...
20200112_中華電信_65531,"[Russia, Russia, Russia, Russia, Russia, Russi...","[1, 2, 2, 2, 2, 2, 2, 1, 2, 3, 1, 1, 2, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200112_中華電信_65532,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200112_中華電信_65533,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200112_中華電信_65534,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."


In [16]:
%%time
def get_port(idx):
    return idx.split('_')[-1]
def get_proto(protocol_li):
    taken = []
    protocol_li_rev = []
    defined_protos = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip','tds',
                 'data','ssl','sock','rtsp','gsm_ipa','nbss','giop','ldap','mptcp','irc','ecatf','mqtt','ams','tfp',
                'amqp','x11','bittorrent','ndps','vnc','h501','dns']
    for x in protocol_li:
        try:
            x = x.split(':')[3:]
        except AttributeError:
            continue
        x  = sorted(list(set(x)),reverse=True)
        protocol_li_rev.append(x)
#     protocol_li_rev = [sorted(list(set(x.split(':')[3:])),reverse=True) for x in protocol_li]
    protocol_li_rev = [" ".join(x) for x in protocol_li_rev]
    for pro_str in protocol_li_rev:
        for proto in defined_protos:
            if proto in pro_str:
                final = proto
                break
            else:
                final = pro_str
        taken.append(final)
    return list(set(taken))
def set2list(li):
    return list(set(li))
port_df['port_num'] = port_df.index.map(get_port)
port_df['in_pkts'] = port_df.session_i_tt_packet.apply(sum)
port_df['out_pkts'] = port_df.session_o_tt_packet.apply(sum)
port_df['in_proto'] = port_df.frame_i_max_protocols.apply(get_proto)
port_df['out_proto'] = port_df.frame_o_max_protocols.apply(get_proto)
port_df['country_set'] = port_df.country.apply(set2list)
port_df['session_num'] = port_df.session_i_tt_packet.apply(len)
port_df

CPU times: user 2h 40min 56s, sys: 5h 43min 7s, total: 8h 24min 3s
Wall time: 20h 8min 12s


,country,session_i_tt_packet,session_o_tt_packet,frame_i_max_protocols,frame_o_max_protocols,port_num,in_pkts,out_pkts,in_proto,out_proto,country_set,session_num
20200106_中嘉寬頻_0,"[Denmark, Netherlands]","[1, 1]","[1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp]",0,2,2,[tcp],[tcp],"[Netherlands, Denmark]",2
20200106_中嘉寬頻_1,"[France, France, Russia, Russia, Russia, Franc...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",1,927,470,[tcp],[tcp],"[Canada, Germany, Singapore, Russia, Thailand,...",470
20200106_中嘉寬頻_2,"[Russia, Russia, Russia, Denmark]","[2, 2, 2, 1]","[1, 1, 1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",2,7,4,[tcp],[tcp],"[Russia, Denmark]",4
20200106_中嘉寬頻_3,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",3,102,52,[tcp],[tcp],"[Russia, Denmark]",52
20200106_中嘉寬頻_4,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",4,475,238,[tcp],[tcp],"[Russia, Denmark]",238
...,...,...,...,...,...,...,...,...,...,...,...,...
20200112_中華電信_65531,"[Russia, Russia, Russia, Russia, Russia, Russi...","[1, 2, 2, 2, 2, 2, 2, 1, 2, 3, 1, 1, 2, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",65531,1497,826,[tcp],[tcp],"[Germany, China, Russia, France, United States...",759
20200112_中華電信_65532,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",65532,1785,957,[tcp],[tcp],"[Russia, France, United States, China]",918
20200112_中華電信_65533,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",65533,1858,1019,[tcp],[tcp],"[China, Saudi Arabia, Russia, France, United S...",993
20200112_中華電信_65534,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",65534,1971,1034,[tcp],[tcp],"[China, Russia, France, United States, Sweden]",1004


In [15]:
# def get_proto(protocol_li):
#     taken = []
#     protocol_li_rev = []
#     defined_protos = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip','tds',
#                  'data','ssl','sock','rtsp','gsm_ipa','nbss','giop','ldap','mptcp','irc','ecatf','mqtt','ams','tfp',
#                 'amqp','x11','bittorrent','ndps','vnc','h501','dns']
#     for x in protocol_li:
#         try:
#             x = x.split(':')[3:]
#         except AttributeError:
#             continue
#         x  = sorted(list(set(x)),reverse=True)
#         protocol_li_rev.append(x)
# #     protocol_li_rev = [sorted(list(set(x.split(':')[3:])),reverse=True) for x in protocol_li]
#     protocol_li_rev = [" ".join(x) for x in protocol_li_rev]
#     for pro_str in protocol_li_rev:
#         for proto in defined_protos:
#             if proto in pro_str:
#                 final = proto
#                 break
#             else:
#                 final = pro_str
#         taken.append(final)
#     return list(set(taken))
# li = ['eth:ethertype:ip:tcp', 'eth:ethertype:ip:tcp','eth:ethertype:ip:tcp:app']
# get_proto(li)

['tcp', 'tcp app']

In [17]:
%%time
port_df_rev = port_df[['port_num','session_num','in_pkts','out_pkts','in_proto','out_proto','country_set']]
port_df_rev_gby = port_df_rev.groupby(['port_num']).agg(list)
def count_country(lioli):
    return len(set(functools.reduce(operator.iconcat, lioli, [])))
port_df_rev_gby['session_num'] = port_df_rev_gby.session_num.apply(sum)
port_df_rev_gby['in_pkts'] = port_df_rev_gby['in_pkts'].apply(sum)
port_df_rev_gby['out_pkts'] = port_df_rev_gby['out_pkts'].apply(sum)
port_df_rev_gby['country_num'] = port_df_rev_gby['country_set'].apply(count_country)
port_df_rev_gby.index = port_df_rev_gby.index.astype(int)
port_df_rev_gby.sort_index(inplace=True)
port_df_rev_gby['session_portion'] = round((port_df_rev_gby['session_num'] / port_df_rev_gby['session_num'].sum())*100,4)
port_df_rev_gby

CPU times: user 12.4 s, sys: 8.89 s, total: 21.3 s
Wall time: 25.8 s


,session_num,in_pkts,out_pkts,in_proto,out_proto,country_set,country_num,session_portion
port_num,,,,,,,,
0,4541,10606,6423,"[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[Netherlands, Denmark], [Netherlands, Estonia...",23,0.0003
1,54301,105773,54214,"[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[Canada, Germany, Singapore, Russia, Thailand...",21,0.0037
2,44920,80836,44976,"[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[Russia, Denmark], [Russia], [Russia], [Russi...",15,0.0031
3,35564,69334,35469,"[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[Russia, Denmark], [Russia], [Russia], [Russi...",18,0.0024
4,32762,64218,32843,"[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[Russia, Denmark], [Russia], [Russia], [Russi...",16,0.0022
...,...,...,...,...,...,...,...,...
65531,30077,59366,30016,"[[tcp, tcp ip icmp], [tcp], [tcp], [tcp], [tcp...","[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[Indonesia, Russia, Netherlands], [Russia], [...",18,0.0021
65532,15628,30549,15463,"[[tcp ip icmp], [tcp], [tcp ip icmp], [tcp], [...","[[], [tcp], [], [tcp], [], [], [tcp], [], [tcp...","[[Russia], [Russia], [Egypt], [Russia], [India...",15,0.0011
65533,57109,112877,93858,"[[tcp], [tcp, tcp ip icmp], [tcp], [tcp], [tcp...","[[tcp], [tcp], [tcp], [tcp], [tcp], [tcp], [tc...","[[India, China], [Albania, Greece, Indonesia, ...",38,0.0039


In [ ]:
port_df_rev_gby_save = port_df_rev_gby[['session_num','session_portion','in_pkts','out_pkts','country_num']]
port_df_rev_gby_save.to_excel('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pictures/nopayload_dist.xlsx')